## 强化学习和马尔可夫链 第一部分
### 强化学习简介
&emsp;&emsp;本系列文章的目的是帮助大家认识强化学习，了解强化学习的基础算法和一些简单的应用。首先我个人觉得任何知识或者方法的提出必定是有对应的时代背景和当时的问题存在，才会有土壤去催生一种新的方法论，那么不妨让我在本系列的第一篇文章里浅浅的对强化学习有个框架式的介绍。

&emsp;&emsp;强化学习的思想，源于行为心理学的研究。1911年Thorndike提出了效用法则(Law of Effect):一定情形下让动物感到舒服的行为，就会与此情景加强联系，动物的这种行为也会在此情景重现时更易发生；相反，那么让动物不舒服的行为也会减弱它们与该情景的联系。如果我们把“感到舒服”理解为“奖励”，把“不舒服”理解为“惩罚”，那么我们就得到了如下的结论：在给定情景下，得到奖励的行为会被“强化”而收到惩罚的行为就会被“弱化”。这样一种生物智能模式使得动物可以从不同行为尝试获得的奖励或惩罚学会在该情景下选择训练者最期望的行为。**这就是强化学习的核心机制：用试错(trail-and-error)来学会在给定的情境下选择最恰当的行为。**

&emsp;&emsp;接下来我们就开始正式介绍强化学习。**强化学习(Reinforcement learning,RL)讨论的问题是智能体(agent)怎么在复杂，不确定的环境(environment)里面去最大化它的奖励。**如图所示，强化学习由两部分组成：智能体和环境。在强化学习过程中，智能体与环境一直在交互。智能体在环境里面获取某个状态后，它会利用该状态输出一个动作（action），这个动作也称为决策（decision）。然后这个动作会在环境之中被执行，环境会根据智能体采取的动作，输出下一个状态(observation)以及当前这个动作带来的奖励。智能体的目的就是尽可能多地从环境中获取奖励。

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;![Alt text](图1.jpg)
&emsp;&emsp;正如上文我们提到的，强化学习的目标就是选取一系列的动作来最大化奖励，所有选取的动作都有可能有长期的影响。但在这个过程里面，智能体的奖励其实是被延迟了的，就是我们现在选取的某一步动作，可能要等到很久后才知道这一步到底产生了什么样的影响。一个最简单的例子就是我们高中三年的学习都是为了在高考时取得好的成绩，但是我们日常的努力并不会立即得到反馈，而是要到考场上才会显现出来。

&emsp;&emsp;在与环境的交互过程中，智能体会获得很多**观测**。针对每一个观测，智能体会采取一个动作，也会得到一个奖励。所以智能体的行为历史是一段关于观测、动作、奖励的序列：$$H_{t}=o_{1},a_{1},r_{1},...,o_{t},a_{t},r_{t}$$

### 状态空间(observation space)
&emsp;&emsp;智能体在采取当前动作的时候会依赖于它之前得到的历史。所以我们可以把整个游戏的状态看到关于这个历史的函数:$$s_{t}=f(H_{t})$$

&emsp;&emsp;在这里我们要稍微暂停一下，因为在上文中我们出现了两个概念，一个是**状态**一个是**观测**，这两个概念看起来是相似的，但其实有很大的不同。**状态**指的是对当前环境的完整的描述，不会对环境的信息做隐藏。而**观测**是智能体所观察的状态，可能会遗漏一些信息。所以，通俗一点来说，**状态是由环境更新的，而观测是由智能体更新的，观测永远被包含在状态之中。**所以当智能体的观测和环境的状态等价时，即当智能体能够观察到环境的所有状态时，我们称这个环境是完全可观测的(fully observed)。那当它们不等价的时候，此时我们就称环境是部分可观测的。
### 动作空间(action space)
&emsp;&emsp;不同的环境允许不同种类的动作。在给定的环境中，有效动作的集合经常被称为**动作空间(action space)**。像下棋或者走迷宫的环境中，动作是离散的，此时我们就可以说这样的环境有**离散动作空间(discrete action space)**,在这个动作空间里，智能体的动作数量是有限的。在其它环境中，比如在物理世界中控制一个智能体，这个环境中就有**连续动作空间(continuous action space)**。在连续动作空间中，动作是实值的向量。例如，在自动驾驶中方向盘的旋转角度就是一个连续动作，其动作空间就为连续动作空间。
### 智能体的组成
&emsp;&emsp;刚才我们介绍了状态空间和动作空间，但我们可以发现这两个因素都不是智能体能够决定的，而是智能体所处的环境决定的。那么我们接下来将讨论关于智能体自身的特性。
- **策略（policy）**：智能体会用策略选取下一步的动作。
- **价值函数（value function）**：我们用价值函数来对当前状态进行评估。价值函数用于评估智能体进入某个状态后，可以对后面的奖励带来多大的影响。价值函数越大，说明智能体进入这个状态越有利。

下面我们深入了解这几个组成部分的细节。
#### 策略
&emsp;&emsp;策略是智能体的动作模型，它决定了**智能体选取动作的方式**。它其实是一个函数，用于把输入的状态变成动作，策略可分为两种：随机性策略和确定性策略。
&emsp;&emsp;**随机性策略(stochastic policy)**就是$\pi$函数，即$\pi(a|s)=p(a_{t}=a|s_{t}=s)$。输入一个状态，输出一个概率。这个概率是智能体所有动作的概率，然后对这个概率分布进行采样，可得到智能体将采取的动作。比如可能是有0.7的概率往左，0.3的概率往右，那么通过采样就可以得到智能体将采取的动作。
&emsp;&emsp;**确定性策略(deterministic policy)**就是用贪心的方式直接采取概率最大的动作，即$a = \mathop{\arg\max}\limits_{a}\pi(a|s)$。
&emsp;&emsp;通常情况下，强化学习一般使用随机性策略，随机性策略有很多优点。比如，在学习时可以通过引入一定的随机性来更好地探索环境；随机性策略的动作具有多样性，这一点在多个智能体博弈时非常重要。采用确定性策略的智能体总是对同样的状态采取相同的动作，非常容易陷入局部最优。
#### 价值函数
&emsp;&emsp;价值函数的值是对未来奖励的预测，我们用它来评估状态的好坏。价值函数里面有一个**折扣因子$\gamma$(discount factor)**，折扣因子表示我们对未来奖励的重视程度。
&emsp;&emsp;正如我们上文所说，很多时候奖励是具有延迟性的，比如我今天写了这篇博客，可能要过很长时间才能被许多人看见，这就是延迟奖励，而我写完博客的即时奖励可能就只有自己的成就感。**我们会发现好像延迟奖励也很重要，但是真实情况是，往往即时奖励，即一些即时的正反馈才是驱使动作的最大因素**，所以我们往往将$\gamma$设置为0.99或者0.95等较接近为1的数值，这样我们的延迟奖励就不会像即时奖励那样重要，但在价值函数中也占了不少的比重。我们的价值函数定义如下：$$V_{\pi}=\mathbb{E}[G_{t}|s_{t}=s]=\mathbb{E_{\pi}}[{\sum_{k=0}^{\infty}\gamma^{k}r_{t+k+1}|s_{t}=s}]$$
&emsp;&emsp;期望$\mathbb{E}_{\pi}$的下标是$\pi$函数，$\pi$函数的值可反映在我们使用策略$\pi$的时候，在每个时刻到底可以得到多少奖励。
&emsp;&emsp;我们还有一种价值函数:$Q$函数,（$Q$函数会在接下来用专门的章节去介绍）。$Q$函数里面包含两个变量:状态和动作。其定义为$$Q_{\pi}(s,a)=\mathbb{E}_{\pi}[G_{t}|s_{t}=s,a_{t}=a]=\mathbb{E}_{\pi}[{\sum_{k=0}^{\infty}\gamma^{k}r_{t+k+1}|s_{t}=s},a_{t}=a]$$

&emsp;&emsp;在了解策略和价值函数之后，我们不妨来通过一个简单的例子来阐明一下这两个概念。
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;![Alt text](图2.jpg)

&emsp;&emsp;如图所示，我们要求智能体从起点开始，然后到达终点的位置。每走一步，环境给我们反馈的奖励是-1。我们可以采取的动作是往上、下、左、右走。我们用现在智能体所在的位置来描述当前状态。
&emsp;&emsp;我们可以用不同的强化学习方法来解这个环境。如果我们采取基于策略的强化学习（policy-based RL）方法，当学习好了这个环境后，在每一个状态，我们都会得到一个最佳的动作。如图所示，比如我们现在在起点位置，我们知道最佳动作是往右走；在第二格的时候，得到的最佳动作是往上走；第三格是往右走...... 通过最佳的策略，我们可以最快地到达终点。

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;![Alt text](图3.jpg)

&emsp;&emsp;如果换成基于价值的强化学习（value-based RL）方法，利用价值函数作为导向，我们就会得到另外一种表征，每一个状态会返回一个价值。如图所示，比如我们在起点位置的时候，价值是 −16，因为我们最快可以 16 步到达终点。因为每走一步会减 1，所以这里的价值是 −16。当我们快接近终点的时候，这个数字变得越来越大。在拐角的时候，比如现在在第二格，价值是 −15，智能体会看上、下两格，它看到上面格子的价值变大了，变成 −14 了，下面格子的价值是 −16，那么智能体就会采取一个往上走的动作。所以通过学习的价值的不同，我们可以抽取出现在最佳的策略。
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;![Alt text](图4.jpg)
### 基于价值的智能体与基于策略的智能体
&emsp;&emsp;根据智能体学习的事物不同，我们可以把智能体进行归类。基于价值的智能体（value-based agent）显式地学习价值函数，隐式地学习它的策略。策略是其从学到的价值函数里面推算出来的。基于策略的智能体（policy-based agent）直接学习策略，我们给它一个状态，它就会输出对应动作的概率。基于策略的智能体并没有学习价值函数。把基于价值的智能体和基于策略的智能体结合起来就有了演员-评论员智能体（actor-critic agent）（后面会详细介绍）。这一类智能体把策略和价值函数都学习了，然后通过两者的交互得到最佳的动作。
### 强化学习的探索和利用
&emsp;&emsp;在强化学习里面，探索和利用是两个很核心的问题。探索即我们去探索环境，通过尝试不同的动作来得到最佳的策略（带来最大奖励的策略）。利用即我们不去尝试新的动作，而是采取已知的可以带来很大奖励的动作。在刚开始的时候，强化学习智能体不知道它采取了某个动作后会发生什么，所以它只能通过试错去探索，所以探索就是通过试错来理解采取的动作到底可不可以带来好的奖励。利用是指我们直接采取已知的可以带来很好奖励的动作。所以这里就面临一个权衡问题，即怎么通过牺牲一些短期的奖励来理解动作，从而学习到更好的策略。
&emsp;&emsp;举一个简单的例子，比如今天我去食堂打饭，我点了一份糖醋排骨，觉得非常好吃。那么我为了能够让我每天中午都吃的开心一点，我可能每一天的荤菜都点糖醋排骨，这就属于**利用(exploitation)**，我不断采取目前已知的可以获得最大奖励的动作。但是有一天我去食堂的时候我发现糖醋排骨旁边的土豆烧肉看起来也很好吃，我尝了一口发现好像一般般，但是下一次我又尝试了小炒肉，发现比糖醋排骨还要好吃，这就属于**探索(exploration)**,我每次都去做不同的选择，从而找到那个能获得最大奖励的动作。
### python中的gym库
&emsp;&emsp;gym是一个python的第三方库，里面包含了很多较为简单的强化学习环境，我们今天就拿$Cartpole$举例子。


In [ ]:
!pip install gym #在terminal中输入pip指令就可以下载

 <img src="图5.jpg" width="800">

In [ ]:
#我们可以在代码打印出动作空间和观测空间。
import gym
env = gym.make('CartPole-v0')#导入环境时名字一定要输对
env.reset()#在新一轮游戏开始之前要重置游戏
for step in range(1000):
	print(env.action_space)#打印动作空间
	action = env.action_space.sample() #随机从动作空间中选取一个动作
	observation, reward, done, info = env.step(action) #在当前状态采取该动作之后会得到的奖励和新的状态，done用来表示游戏是否已经结束，如果结束就会开始一个新的回合
	print(observation)
'''
#输出
Discrete(2)
[ 0.01653398 0.19114579 0.02013859 -0.28050058]
可以看到我们的动作状态空间是包含了两个动作的离散状态空间，并且状态值由四个值构成。
'''